In [1]:
pip install ucimlrepo

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from ucimlrepo import fetch_ucirepo 
  
# Aquí se hace fecth al dataset de Abalone
abalone = fetch_ucirepo(id=1) 
  
# data (as pandas dataframes) 
X = abalone.data.features 
y = abalone.data.targets 


In [3]:
import numpy as np

#Junto la columna de targets a la de features para hacer X_full
X_full = X.join(y)

#Inicializo la lista de atípicos
total_atipicos=[]
for column in X_full:
    #Exluyo la columna cualitativa
    if column == "Sex":
        continue

    #Calculo los cuartiles
    x_temp = X_full[column]
    q3=np.quantile(x_temp, 0.75)
    q1=np.quantile(x_temp, 0.25)
    limsp=q3+1.5*(q3-q1)
    liminf=q1-1.5*(q3-q1)
    for i in range(0, len(x_temp)):
        #Si el valor es mayor al límite superior o menor al límite inferior, lo agrego a la lista de atípicos
        if x_temp[i]>limsp:
            if i not in total_atipicos:
                total_atipicos.append(i)
        if x_temp[i]<liminf:
            if i not in total_atipicos:
                total_atipicos.append(i)    

In [4]:
#Elimino los atípicos y reseteo el índice
X_new = X_full.drop(total_atipicos)
X_new = X_new.reset_index()

In [5]:
X_new

,index,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings
0,0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15
1,1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7
2,2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9
3,3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10
4,4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7
...,...,...,...,...,...,...,...,...,...,...
3776,4172,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
3777,4173,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
3778,4174,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
3779,4175,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


In [6]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

#Se declara el modelo de regresión lineal
regr = linear_model.LinearRegression()

In [7]:
#Se elimina la columna de índices sobrante
X_new=X_new.drop("index", axis="columns")

In [8]:
import itertools
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

#Función que recibe una columna y un dataframe, y devuelve el mejor modelo de regresión lineal según el mae, mape y mse
def regression_model_combs (column, df):
    #Excluyo la columna cualitativa y la columna de la salida
    excluded_cols = ["Sex", column]
    included_cols = [col for col in df.columns if col not in excluded_cols]

    total_metrics = []

    #Realizo combinaciones de 1 a 4 columnas
    for combination in range(1,5):
        for combo in itertools.combinations(included_cols, combination):
            #Se hace una copia del dataframe para no modificar el original
            X1 = df.copy()
            input=X1[list(combo)]

            #Se ajusta el modelo de regresión lineal
            regr.fit(input, X1[column])
            Y_predict=(regr.predict(input))
            X1['predict']=Y_predict

            #Se calculan las métricas de MAE, MAPE y MSE
            mae = mean_absolute_error(X1[column], X1['predict'])
            mape = mean_absolute_percentage_error(X1[column], X1['predict'])
            mse = mean_squared_error(X1[column], X1['predict'])

            #Se guardan las métricas en una lista
            total_metrics.append({
                "name": f'{combo}',
                "mae": mae,
                "mape": mape,
                "mse": mse,
            })

    sorted_mae = sorted(total_metrics, key=lambda x: x['mae'])
    print(column,": Entrada: ", sorted_mae[0]["name"], "MAE: ", sorted_mae[0]["mae"])
    sorted_mape = sorted(total_metrics, key=lambda x: x['mape'])
    print(column, ": Entrada: ", sorted_mape[0]["name"], "MAPE: ", sorted_mape[0]["mape"])
    sorted_mse = sorted(total_metrics, key=lambda x: x['mse'])
    print(column, ": Entrada: ", sorted_mse[0]["name"], "MSE: ", sorted_mse[0]["mse"])

In [9]:
#Hago las combinaciones con los datos atípicos eliminados
combinations_metrics=regression_model_combs("Rings", X_new)

Rings : Entrada:  ('Diameter', 'Height', 'Shucked_weight', 'Shell_weight') MAE:  1.3008215842511464
Rings : Entrada:  ('Diameter', 'Height', 'Shucked_weight', 'Shell_weight') MAPE:  0.14127322492327296
Rings : Entrada:  ('Diameter', 'Height', 'Shucked_weight', 'Shell_weight') MSE:  2.80755371234152


In [10]:
#Hago las combinaciones con los datos atípicos incluidos
combinations_metrics2=regression_model_combs("Rings", X_full)

Rings : Entrada:  ('Diameter', 'Whole_weight', 'Shucked_weight', 'Viscera_weight') MAE:  1.633344713236118
Rings : Entrada:  ('Diameter', 'Height', 'Shucked_weight', 'Shell_weight') MAPE:  0.16399001943124217
Rings : Entrada:  ('Diameter', 'Whole_weight', 'Shucked_weight', 'Shell_weight') MSE:  5.040553730396935


Se puede observar que, al realizar el modelo de regresión con todas las combinaciones posibles para un máximo de 4 entrada, el mejor modelo es el ('Diameter', 'Height', 'Shucked_weight', 'Shell_weight') con un MAPE de 0.14127322492327296 que significa un 14.12% de error, esto para un modelo sin datos atípicos. No obstante, para el caso de datos atípicos, esta combinación de entradas es la mejor igualmente con un MAPE de 0.16399001943124217.

Para la métrica del MAE, el mejor modelo, tanto para el set de atípicos como para el de sin atípicos, es ('Diameter', 'Height', 'Shucked_weight', 'Shell_weight') con un valor de 1.3008215842511464, la cual es la misma combinación de entradas anteriormente mencionada. Tal caso ocurre de la misma forma para el MSE que, con las mismas entradas y sin datos atípicos, logró un valor de 2.80755371234152.

En coclusión, el mejor modelo para predecir los anillos de los abalones es la combinación de entradas ('Diameter', 'Height', 'Shucked_weight', 'Shell_weight'), excluyendo los datos atípicos.

In [12]:
X_final = X_new[["Diameter", "Height", "Shucked_weight", "Shell_weight"]]

In [13]:
correlation_matrix = X_final.corr()
correlation_matrix

,Diameter,Height,Shucked_weight,Shell_weight
Diameter,1.000000,0.900426,0.909477,0.929413
Height,0.900426,1.000000,0.849048,0.901294
Shucked_weight,0.909477,0.849048,1.000000,0.903340
Shell_weight,0.929413,0.901294,0.903340,1.000000


Asimismo, observando la matriz de correlación, se puede observar que las columnas tienen una correlaci en un mínimo de 0.849048 para Height y Shucked_weight, y un máximo de 0.929413 para Diameter y Shell_weight.